In [1]:
import re
import torch.nn.functional as F
from torch import Tensor
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from time import gmtime, strftime

C:\Users\ChangGeng\anaconda3\envs\torchi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large').to(device)

In [4]:
df = pd.read_json("../Dataset/DataCleanedDF.json")

In [5]:
df.head()

,Path,Text
1,D:/Computer/Benutzer/Computerwelt/IEEE2022 Bac...,Form W-8BEN(RevOctober 2021)Department of th...
2,D:/Computer/Benutzer/Computerwelt/Instagrampos...,"Skript zur VorlesungDatenstrukturen\""ProfDrGeo..."
3,D:/Computer/Benutzer/Computerwelt/Instagrampos...,"Perl, PHP, PythonEin VergleichVor- und Nachtei..."
4,D:/Computer/Benutzer/Computerwelt/Kaggle/Cheet...,Python Cheat Sheet: Keywords “​A puzzle a day ...
5,D:/Computer/Benutzer/Computerwelt/Kaggle/Cheet...,"Boolean Integer, Float String Python Cheat She..."


In [6]:
# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: data science',
               "passage: Wir benutzen einen vortrainierten K-Nearest-Neighbour-Classifier um unsere Testdaten zu labeln.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]


In [7]:
texts = df["Text"].tolist()
input_texts = []
for text in texts:
    input_texts.append("passage: ")
    for line in text.split("\n"):
        input_texts[-1] += line
        if len(input_texts[-1])>1000:
            input_texts.append("passage: ")

In [8]:
len(input_texts)

3097

In [9]:
def embed(input_texts):
    model.eval()
    # Tokenize the input texts
    batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)

    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

        # normalize embeddings
        embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings

In [10]:
batch_size = 400
embeddings = []
for i in range(0,len(input_texts),batch_size):
    embeddings.append(embed(input_texts[i:i+batch_size]).cpu())
    # print(i)
    print(f"{i*100/len(input_texts)}: {i} von {len(input_texts)}")
    
    
embeddings = torch.cat(embeddings)

0.0: 0 von 3097
12.915724895059736: 400 von 3097
25.83144979011947: 800 von 3097
38.7471746851792: 1200 von 3097
51.66289958023894: 1600 von 3097
64.57862447529868: 2000 von 3097
77.4943493703584: 2400 von 3097
90.41007426541815: 2800 von 3097


In [11]:
zeit = strftime("%Y_%m_%d_%a_%H_%M_%S", gmtime()).replace(" ", "_")

In [12]:
zeit = strftime("%Y_%m_%d_%a_%H_%M_%S", gmtime()).replace(" ", "_")
torch.save(embeddings,f"CGD's Google_{zeit}.pt")
embeddings = torch.load(f"CGD's Google_{zeit}.pt")

In [13]:
#########################################################################
'''
================================== Alt ==================================
query_text = input("SUCHE: ")
query_embedding = embed(["query: "+query_text]).cpu()
scores = (query_embedding @ embeddings.T)[0] * 100
flag = 0
for score,text in sorted(zip(scores,input_texts),reverse=True)[:10]:
    if len(text) >= 200:
        print(score.item())
        print(text)
        flag += 1
        print(flag)
        print()
'''

'\n================================== Alt ==================================\nquery_text = input("SUCHE: ")\nquery_embedding = embed(["query: "+query_text]).cpu()\nscores = (query_embedding @ embeddings.T)[0] * 100\nflag = 0\nfor score,text in sorted(zip(scores,input_texts),reverse=True)[:10]:\n    if len(text) >= 200:\n        print(score.item())\n        print(text)\n        flag += 1\n        print(flag)\n        print()\n'

In [27]:
query_text = input("SUCHE: ")
query_embedding = embed(["query: " + query_text]).cpu()
scores = (query_embedding @ embeddings.T)[0] * 100
print("\n")

flag = 0
for score, text in sorted(zip(scores, input_texts), reverse=True):
    if len(text) >= 200 and flag < 10:
        df_path = df["Path"][df["Text"].str.contains(text[20:50])]
        print(list(zip(df_path.index, df_path.values))) # text.replace("passage: ", "") # text[-19:] -> funktioniert
        
        print(score.item())
        print(text)
        flag += 1
        print(flag)
        print("========="*40)
        print()
        print()
        if flag == 10:
            break

SUCHE:  Physik




[(1155, 'D:/Computer/Benutzer/Uni/FAU/Module/Semester 03/Stochastische Modelbildung/Vorlesung/Vorlesungsaufzeichnungen_02.Nov.pdf')]
83.24552154541016
passage: Vwbscmpbgiom8:30am biden Tajen .Literatur :Hans - Otto GeorgiiU Krengel : Erinfihrwng in die Wahrsdeintidhib -Stodastih , Itufkge:theorieand statistics , 8Aufkge .EinfihmyinStodastih- Fohus : Byñtfe and Methodius do Stodasih( tntopretation und Anyang damnit)↳ FolgeroonsfdtmyCwohrsdinhdhib -theories- Fohus : Mathematik Anyang not dim ftoft(Ariane , Definitionen ,ttneorome / Setu ,Beweise )waisthlahndeinbdhét?•von W'hit .tndnitioesrert-nd.msEregnis A → DCA) C- [oil] W'kaif von A( Map toA eintritt )unser kilobit / Unsidwhét classPCA)=0 :④ (A) = 1P( A) E 10,1) ' Etna dazwislen .Sidersilo , dam A eintritt, dam A nidteinttilt:• Weilerinterpretation :wiedetholenWirmehrer MoleeinRelative Aaifjhét2-ufdlseapoimentlunobhénjg enter gluten Bediyunjn)# { &gwimmk in demon A beobedtet wird* I Gesemttchldednrdgefihrten ExperimentalPCA) ≈Prakti